# Preprocessing .mat for NWB

In [1]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import convert_to_nwb_for_PB
import converters.Initiation_nwb
import scipy.io
import warnings
from datetime import timedelta
from tqdm import tqdm



# Load data 

## .mat files

In [2]:

mat_file = "/Users/lorisfabbro/Documents/MATLAB/PB/mPFC_Preprocessed.mat"
output_folder = "data"

### General

INFO :
- Acceder keys h5py.file :  list(group.keys())
- Lire dataset [()] : roup = f['Data_Full']


In [ ]:
# Cas 1 : codes ASCII (uint8/uint16) -> "LB010"
if arr.dtype.kind in 'iu':
    name = ''.join(chr(int(x)) for x in arr.ravel() if int(x) > 0)
    print("first")
# Cas 2 : tableau de bytes b'...' (dtype 'S')
elif arr.dtype.kind == 'S':
    name = b''.join(arr.ravel().tolist()).decode('utf-8', errors='ignore')
    print("second")
# Cas 3 : unicode (dtype 'U')
elif arr.dtype.kind == 'U':
    name = ''.join(arr.ravel().tolist())
    print("third")
# Cas 4 : valeur unique bytes/str
elif isinstance(val, bytes):
    name = val.decode('utf-8', errors='ignore')
    print("fourth")
elif isinstance(val, str):
    name = val
    print("fifth")
else:
    name = str(val)
    print("sixth")

In [29]:
with h5py.File(mat_file, 'r') as f:
    General_data = f['Data_Full']
    print(list(General_data.keys()))

['Cell_Counter', 'Cell_Depth', 'Cell_ID', 'Cell_Location', 'Cell_Performance', 'Mouse_DateOfBirth', 'Mouse_Genotype', 'Mouse_Name', 'Mouse_Sex', 'Session_Counter', 'Session_Type', 'Sweep_AP_Times', 'Sweep_Behavior', 'Sweep_Counter', 'Sweep_CurrentMonitor', 'Sweep_CurrentMonitor_SamplingRate', 'Sweep_Lick', 'Sweep_Lick_Power', 'Sweep_Lick_SamplingRate', 'Sweep_Lick_Threshold', 'Sweep_Licking_Times', 'Sweep_MembranePotential', 'Sweep_MembranePotential_SamplingRate', 'Sweep_Quiet_Times', 'Sweep_Reward_Times', 'Sweep_StartTime', 'Sweep_Stim_Amp', 'Sweep_Stim_Dur', 'Sweep_Stim_Onset_Times', 'Sweep_Stim_Type', 'Sweep_Subthreshold', 'Sweep_Trial_Onset_Times', 'Sweep_Type', 'Sweep_WhiskerAngle', 'Sweep_WhiskerAngle_SamplingRate', 'Sweep_Whisking_Times']


In [13]:

choice_mouses = ["LB010", "LB011", 'LB012', 'LB039', 'LB042', 'LS001']
mat_file = mat_file
#--------


columns = ['Mouse Name', 'User (user_userName)', 'Cell_ID', 'Ear tag',
    'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
    'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
    'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
    'Session Type','sweeps', 'Opto Session', 'Mouse Age (d)', 'Weight of Reference',
    'Weight Session']
csv_data = pd.DataFrame(columns=columns)
csv_data.columns = csv_data.columns.str.strip()

def to_text(x):
    a = np.array(x)
    if isinstance(x, (bytes, np.bytes_)):
        return x.decode('utf-8', errors='ignore')
    if isinstance(x, (str, np.str_)):
        return str(x)

    if a.dtype.kind in ('i','u'):
        return ''.join(chr(int(v)) for v in a.ravel() if int(v) > 0)

    if a.dtype.kind == 'S':
        return b''.join(a.ravel().tolist()).decode('utf-8', errors='ignore')

    if isinstance(x, (np.integer, int)):
        return chr(int(x))

    return str(x)

def read_cell_numeric_vectors(f, ref_array):
    """Return np.array for each cell"""
    out = []
    for r in np.ravel(ref_array):
        obj = f[r]
        val = np.array(obj[()])
        out.append(val.squeeze())
    return out

def read_cell_strings(f, ref_array):
    """Return a list of strings (or None if empty)."""
    out = []
    for r in np.ravel(ref_array):
        obj = f[r]
        val = np.array(obj[()])
        if val.size == 0:
            out.append(None)
        elif val.dtype.kind in ('S', 'U'):  
            s = val.tobytes().decode('utf-8', errors='ignore') if val.dtype.kind == 'S' else str(val)
            out.append(s.strip() if s else None)
        else:
            try:
                chars = ''.join(chr(int(x)) for x in val.ravel() if int(x) > 0)
                out.append(chars if chars else None)
            except Exception:
                out.append(None)
    return out

def safe_abs_times(rel_times, t0):
    """Relative times (s) -> absolute times (datetime) from t0.
    Handles None/NaN/empty arrays."""
    if rel_times is None or (isinstance(rel_times, float) and np.isnan(rel_times)):
        return None
    arr = np.atleast_1d(rel_times).astype(float)
    if arr.size == 0:
        return []
    return [t0 + timedelta(seconds=float(x)) for x in arr]

def safe_list(x):
    if x is None:
        return []
    arr = np.atleast_1d(x)
    return arr.tolist()

def _decode_chars(x):
    a = np.array(x)
    return ''.join(chr(int(v)) for v in a.ravel() if int(v) > 0)

def read_cell_strings(f, path):
    refs = f[path][()]
    return np.array([_decode_chars(f[r][()]) for r in np.ravel(refs)])

def read_birth_dates(f, path):
    refs = f[path][()]
    out = []
    for r in np.ravel(refs):
        vals = [int(f[r2][()][()]) for r2 in np.ravel(f[r][()])]
        # format dd.mm.yy (même ordre que ton code)
        out.append(f"{vals[2]:02d}.{vals[1]:02d}.{vals[0]}")
    return np.array(out)

def read_sweep_start_times(f, path):
    ST = np.array(f[path][()])  # shape (6, N) en général
    times = []
    for raw in ST.T:
        raw = np.array(raw, dtype=float)
        if np.all(np.isnan(raw)): 
            times.append(np.nan); continue
        y, m, d, H, M, S = map(int, raw[:6])
        if S == 60: S, M = 0, M + 1
        times.append(datetime(y, m, d, H, M, S))
    return np.array(times)

def _decode_any(x):
    """Décoder string MATLAB depuis divers formats (bytes, char-codes, nested)."""
    a = np.array(x)
    if a.size == 0:
        return None
    # déjà string/bytes
    if a.dtype.kind == 'S':
        return a.tobytes().decode('utf-8', errors='ignore').strip()
    if a.dtype.kind == 'U':
        return str(a).strip()
    # tableau numérique de codes ASCII
    if a.dtype.kind in ('i', 'u'):
        return ''.join(chr(int(v)) for v in a.ravel() if int(v) > 0)
    # scalaire bytes/str
    if isinstance(x, (bytes, np.bytes_)):
        return x.decode('utf-8', errors='ignore').strip()
    if isinstance(x, (str, np.str_)):
        return x.strip()
    return None

def read_cell_strings_from_refs(f, ref_array):
    """Supporte: string simple OU cell-array de strings (références imbriquées)."""
    out = []
    for r in np.ravel(ref_array):
        obj = f[r][()]
        arr = np.array(obj, dtype=object)

        # Cas 1: cell-array -> éléments peuvent être des références
        if arr.dtype == object:
            items = []
            for el in arr.ravel():
                if isinstance(el, h5py.h5r.Reference):
                    items.append(_decode_any(f[el][()]))
                else:
                    items.append(_decode_any(el))
            # nettoie et filtre None/vide
            items = [s for s in items if s not in (None, '', ' ')]
            out.append(items if len(items) > 1 else (items[0] if items else None))
        else:
            # Cas 2: string direct / char-codes
            out.append(_decode_any(arr))
    return np.array(out, dtype=object)

with h5py.File(mat_file, 'r') as f:

    Cell_ID        = read_cell_strings(f, 'Data_Full/Cell_ID')
    mouses_name    = np.array([s.split('_')[0] for s in Cell_ID])
    birth_dates    = read_birth_dates(f, 'Data_Full/Mouse_DateOfBirth')
    Mouse_Sex      = read_cell_strings(f, 'Data_Full/Mouse_Sex')
    sessiontypes   = read_cell_strings(f, 'Data_Full/Session_Type')
    behaviortypes  = read_cell_strings(f, 'Data_Full/Sweep_Type')

    # --- SWEEPS ----
    # Start times
    sweep_start_times = read_sweep_start_times(f, 'Data_Full/Sweep_StartTime')

    # Signaux
    mp_refs   = f['Data_Full/Sweep_MembranePotential'][()]
    cm_refs   = f['Data_Full/Sweep_CurrentMonitor'][()]
    wa_refs   = f['Data_Full/Sweep_WhiskerAngle'][()]

    vm_list   = read_cell_numeric_vectors(f, mp_refs)  # 487 éléments, np.array par sweep
    cur_list  = read_cell_numeric_vectors(f, cm_refs)
    wa_list   = read_cell_numeric_vectors(f, wa_refs)

    # AP times
    ap_refs   = f['Data_Full/Sweep_AP_Times'][()]
    ap_list   = read_cell_numeric_vectors(f, ap_refs)

    # Stim
    stim_on_refs = f['Data_Full/Sweep_Stim_Onset_Times'][()]
    stim_amp_refs= f['Data_Full/Sweep_Stim_Amp'][()]
    stim_dur_refs= f['Data_Full/Sweep_Stim_Dur'][()]
    stim_type_refs = f['Data_Full/Sweep_Stim_Type'][()]

    stim_on_list  = read_cell_numeric_vectors(f, stim_on_refs)
    stim_amp_list = read_cell_numeric_vectors(f, stim_amp_refs)
    stim_dur_list = read_cell_numeric_vectors(f, stim_dur_refs)
    stim_type_list = read_cell_strings_from_refs(f, stim_type_refs)

    # Rewards
    reward_refs = f['Data_Full/Sweep_Reward_Times'][()]
    reward_list = read_cell_numeric_vectors(f, reward_refs)

    # Behavior matrix (n_trials x 5)
    behav_refs = f['Data_Full/Sweep_Behavior'][()]
    behav_list = []
    for r in np.ravel(behav_refs):
        mat = np.array(f[r][()]).squeeze()
        if mat.size == 0:
            behav_list.append(np.empty((0,5)))
        else:
            if mat.ndim == 1:
                if mat.size % 5 == 0:
                    mat = mat.reshape((-1,5))
                else:
                    mat = np.expand_dims(mat, 0) if mat.size == 5 else np.empty((0,5))
            behav_list.append(mat)
    sr_vm_cm = 20000
    sr_wa    = 200

# Verify if choice_mouses is correct
if choice_mouses is None:
    choice_mouses = np.unique(mouses_name)
else:
    missing = [name for name in choice_mouses if name not in np.unique(mouses_name)]
    if missing:
        raise ValueError(f"Mouse name(s) not found in csv file: {missing}")
    
index_mouses_choices = [np.where(mouses_name == mouse)[0] for mouse in choice_mouses if mouse in mouses_name]
index_mouses_choices = np.concatenate(index_mouses_choices)


# Indices Session (cell_ID) for NWB
unique_ids, indice_unique_cell_id = np.unique(Cell_ID, return_index=True)
cell = [np.arange(len(Cell_ID))[Cell_ID == unique_id] for unique_id in unique_ids]
# Itèration over the indices of the choice mouses to collect data in the dataframe
for one_cell in tqdm(cell, total=len(cell), desc="Cellules"):
    if one_cell[0] in index_mouses_choices:
        name = mouses_name[one_cell[0]]
        user = name[:2]
        birth_date = birth_dates[one_cell[0]]
        sex = Mouse_Sex[one_cell[0]]
        session_type = sessiontypes[one_cell[0]]
        behaviortype = " & ".join(np.unique(behaviortypes[one_cell]))
        start_date = sweep_start_times[one_cell[0]].strftime("%d.%m.%y") if not pd.isnull(sweep_start_times[one_cell[0]]) else None
        session_date = sweep_start_times[one_cell[0]].strftime("%Y%m%d") if not pd.isnull(sweep_start_times[one_cell[0]]) else None
        start_time_hhmmss = sweep_start_times[one_cell[0]].strftime("%H%M%S") if not pd.isnull(sweep_start_times[one_cell[0]]) else None
        end_date = sweep_start_times[one_cell[-1]].strftime("%d.%m.%y") if not pd.isnull(sweep_start_times[one_cell[-1]]) else None

        sweeps = []
        for one_sweep in one_cell:
            t0 = sweep_start_times[one_sweep] if not pd.isnull(sweep_start_times[one_sweep]) else None

            vm      = vm_list[one_sweep]   #if one_sweep < len(vm_list) else np.array([])
            cur     = cur_list[one_sweep]  #if one_sweep < len(cur_list) else np.array([])
            wa      = wa_list[one_sweep]   #if one_sweep < len(wa_list) else np.array([])
            ap_rel  = ap_list[one_sweep]   #if one_sweep < len(ap_list) else np.array([])
            stim_on = stim_on_list[one_sweep] #if one_sweep < len(stim_on_list) else np.array([])
            stim_amp= stim_amp_list[one_sweep] #if one_sweep < len(stim_amp_list) else np.array([])
            stim_dur= stim_dur_list[one_sweep] #if one_sweep < len(stim_dur_list) else np.array([])
            reward_rel = reward_list[one_sweep] #if one_sweep < len(reward_list) else np.array([])
            behav   = behav_list[one_sweep].T #if one_sweep < len(behav_list) else np.empty((0,5))
            stim_type = stim_type_list[one_sweep] #if one_sweep < len(stim_type_list) else None

            # durations
            #dur_vm = float(vm.size / sr_vm_cm) if sr_vm_cm and vm.size>0 else None
            #dur_wa = float(wa.size / sr_wa)    if sr_wa and wa.size>0 else None

            # absolute times
            ap_abs = safe_abs_times(ap_rel, t0) if t0 is not None else None
            stim_on_abs  = safe_abs_times(stim_on, t0)  if t0 is not None else None
            reward_abs   = safe_abs_times(reward_rel, t0) if t0 is not None else None

            # Trials -> liste de dicts (col1=ts rel, col2=stim?, col3=lick, col4=reward, col5=response)
            trials = []
            if behav.size > 0:
                ts_rel   = behav[:,0].astype(float)
                has_stim = behav[:,1].astype(bool)
                lick     = behav[:,2].astype(bool)
                reward_b = behav[:,3].astype(bool)
                response = behav[:,4].astype(int)
                
                ts_abs = [t0 + timedelta(seconds=float(x)) for x in ts_rel] if t0 is not None else [None]*len(ts_rel)

                stim_type = [stim_type] if isinstance(stim_type, str) else ([] if stim_type is None else safe_list(stim_type))

                if len(stim_type) != 0: # if not Free Whisking
                    stim_on = safe_list(stim_on)
                    stim_on_abs = stim_on_abs
                    stim_amp = safe_list(stim_amp)
                    stim_dur = safe_list(stim_dur)
                    if len(has_stim[has_stim > 0]) != len(np.array(stim_on)[np.array(stim_on) > 0]): 
                        raise ValueError(f"Mismatch in lengths: has_stim ({len(has_stim[has_stim > 0])}) vs stim_on ({len(safe_list(stim_on))}) for sweep {one_sweep} and mouse {name}")
                    if stim_on_abs is None : 
                        if  len(stim_on) != len(stim_amp) or len(stim_on) != len(stim_dur) or len(stim_on) != len(stim_type):
                            raise ValueError(f"Stimulus data length mismatch: {len(stim_on)} vs {len(stim_on_abs)}, {len(stim_amp)}, {len(stim_dur)}, {len(stim_type)} for sweep {one_sweep} and mouse {name}")
                    else :
                        if len(stim_on) != len(stim_on_abs) or len(stim_on) != len(stim_amp) or len(stim_on) != len(stim_dur) or len(stim_on) != len(stim_type):
                            raise ValueError(f"Stimulus data length mismatch: {len(stim_on)} vs {len(stim_on_abs)}, {len(stim_amp)}, {len(stim_dur)}, {len(stim_type)} for sweep {one_sweep} and mouse {name}")
                                                
                ind_stim = 0
                for k in range(len(ts_rel)):
                    trials.append({
                        "trial_index": int(k),
                        "time_rel_s": float(ts_rel[k]),
                        "time_abs": ts_abs[k],
                        "lick": bool(lick[k]),
                        "reward": bool(reward_b[k]),
                        "response": int(response[k])  # 0 ou 2
                    })
                    if bool(has_stim[k]) == False:
                        trials.append({
                        "stim": {'has_stim': bool(has_stim[k])}
                        })
                    elif bool(has_stim[k]) == True:
                        trials.append({
                            "stim": {'has_stim': bool(has_stim[k]),
                                     'onset_time_rel_s': stim_on[ind_stim] ,
                                        'onset_time_abs': stim_on_abs[ind_stim] if stim_on_abs is not None else None,
                                        'amplitude': stim_amp[ind_stim],
                                        'duration_s': stim_dur[ind_stim],
                                        'type': stim_type[ind_stim] 
                                     }
                        })
                        ind_stim += 1
            if (ind_stim == 0 and len(stim_type) != 0):
                raise ValueError(f"Stimulus index mismatch: {ind_stim} vs {len(stim_on)-1} for sweep {one_sweep} and mouse {name}")
            elif (ind_stim > 0 and (ind_stim != len(stim_type))):
                raise ValueError(f"Stimulus index mismatch: {ind_stim} vs {len(stim_type)-1} for sweep {one_sweep} and mouse {name}")

            sweep_dict = {
                "Sweep Index": int(one_sweep),
                "Sweep Start Time": t0,
                "Sweep Type": str(behaviortypes[one_sweep]),
                'behav': behav,

                # Signaux
                "membrane_potential": {
                    "sampling_rate_Hz": int(sr_vm_cm),
                    #"n_samples": int(vm.size),
                    #"duration_s": dur_vm,
                    "data": vm.tolist() if vm.size>0 else []
                },
                "current_monitor": {
                    "sampling_rate_Hz": int(sr_vm_cm),
                    #"n_samples": int(cur.size),
                    #"duration_s": float(cur.size/sr_vm_cm) if cur.size>0 else None,
                    "data": cur.tolist() if cur.size>0 else []
                },
                "whisker_angle": {
                    "sampling_rate_Hz": int(sr_wa),
                    #"n_samples": int(wa.size),
                    #"duration_s": dur_wa,
                    "data": wa.tolist() if wa.size>0 else []
                },

                # Spikes
                "ap_times": {
                    "relative_s": safe_list(ap_rel),
                    "absolute": ap_abs
                },
                
                # Rewards
                "reward_times": {
                    "relative_s": safe_list(reward_rel),
                    "absolute": reward_abs
                },

                # Trials
                "trials": trials
            }

            sweeps.append(sweep_dict)



        # Create a new row for the session
        new_row = {
            "Mouse Name": name,
            "User (user_userName)": user,
            "Cell_ID": Cell_ID[one_cell[0]],
            "Ear tag": "Unknown", # UNKNOWN
            "Start date (dd.mm.yy)": start_date,
            "End date": end_date,
            "Sex_bin": sex,
            #"strain": strain, # UNKNOWN
            "mutations": "WT",
            "Birth date": birth_date,
            "licence": "1628", # UNKNOWN
            #"DG": "",
            #"ExpEnd": "",
            "Created on": "Unknown", # UNKNOWN
            "Session": name + "_" + session_date,
            "Session Date (yyymmdd)": session_date,
            "Start Time (hhmmss)": start_time_hhmmss,
            "Behavior Type": behaviortype,
            "Session Type": session_type,
            "sweeps": sweeps,
            "Mouse Age (d)": "Unknown", # UNKNOWN
            "Weight of Reference": "Unknown", # UNKNOWN
            "Weight Session": "Unknown" # UNKNOWN
        }

        # Append the new row to the DataFrame
        csv_data = pd.concat([csv_data, pd.DataFrame([new_row])], ignore_index=True)
csv_data.head(5)
#display(csv_data.iloc[0]["sweeps"][1])




/var/folders/wq/xq9nhd_54tb66bmwplt6b_m00000gn/T/ipykernel_6651/1237140147.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  vals = [int(f[r2][()][()]) for r2 in np.ravel(f[r][()])]
Cellules: 100%|██████████| 66/66 [00:11<00:00,  5.63it/s]


,Mouse Name,User (user_userName),Cell_ID,Ear tag,Start date (dd.mm.yy),End date,Sex_bin,strain,mutations,Birth date,...,Session,Session Date (yyymmdd),Start Time (hhmmss),Behavior Type,Session Type,sweeps,Opto Session,Mouse Age (d),Weight of Reference,Weight Session
0,LB010,LB,LB010_S1_R1,Unknown,08.05.18,08.05.18,M,NaN,WT,25.02.2018,...,LB010_20180508,20180508,134938,Free Whisking & Whisker Stim,Naive,"[{'Sweep Index': 0, 'Sweep Start Time': 2018-0...",NaN,Unknown,Unknown,Unknown
1,LB010,LB,LB010_S1_R3,Unknown,08.05.18,08.05.18,M,NaN,WT,25.02.2018,...,LB010_20180508,20180508,152232,Free Whisking & Whisker Stim,Naive,"[{'Sweep Index': 13, 'Sweep Start Time': 2018-...",NaN,Unknown,Unknown,Unknown
2,LB011,LB,LB011_S1_R3,Unknown,26.06.18,26.06.18,M,NaN,WT,19.03.2018,...,LB011_20180626,20180626,153333,Whisker Stim,Trained,"[{'Sweep Index': 18, 'Sweep Start Time': 2018-...",NaN,Unknown,Unknown,Unknown
3,LB012,LB,LB012_S1_R2,Unknown,14.06.18,14.06.18,M,NaN,WT,19.03.2018,...,LB012_20180614,20180614,145545,Whisker Stim,Trained,"[{'Sweep Index': 22, 'Sweep Start Time': 2018-...",NaN,Unknown,Unknown,Unknown
4,LB012,LB,LB012_S1_R3,Unknown,14.06.18,14.06.18,M,NaN,WT,19.03.2018,...,LB012_20180614,20180614,154249,Whisker Stim,Trained,"[{'Sweep Index': 29, 'Sweep Start Time': 2018-...",NaN,Unknown,Unknown,Unknown


# CSV file


In [ ]:
import pandas as pd 
csv_data = pd.read_csv("data/Subject_Session_Selection_PB.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns
display(csv_data)

Index(['Mouse Name', 'User (user_userName)', 'Cell_ID', 'Ear tag',
       'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
       'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
       'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
       'Session Type', 'Opto Session', 'Mouse Age (d)', 'Weight of Reference',
       'Weight Session'],
      dtype='object')

In [ ]:
csv_row = (csv_data.iloc[9])
csv_row

# TOTAL CSV

In [ ]:
import pandas as pd 
csv_data = pd.read_csv("data/Subject_Session_Selection_PB.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns
display(csv_data)

In [ ]:
importlib.reload(converters.Initiation_nwb)
dataframe_csv_file = converters.Initiation_nwb.files_to_dataframe(mat_file = mat_file, choice_mouses = ["LB010"] )
print(dataframe_csv_file.columns)
dataframe_csv_file.head(30)

 # TOTAL _-_-_-_-_-_-_-_
 

In [ ]:

importlib.reload(convert_to_nwb_for_PB)
nwb_path = convert_to_nwb_for_PB.convert_data_to_nwb_PB(mat_file=mat_file, output_folder="data", mouses_name = ["LB010"])

In [46]:
importlib.reload(converters.Initiation_nwb)
converters.Initiation_nwb.remove_nwb_files("data")